In [44]:
## make sure you're connected to database (as of 10/12/17, is hosted on kanefsky)
from __future__ import division

import pymongo as pm
conn = pm.MongoClient(port=20809)

import numpy as np
import matplotlib
from matplotlib import pylab, mlab, pyplot
plt = pyplot
import seaborn as sns

import gridfs
from PIL import Image
import base64


import pandas as pd
import json

from IPython.core.pylabtools import figsize, getfigs

%matplotlib inline
from pylab import *
from numpy import *

# append to path
sys.path.append('./sketches')
from preprocessing_utils import *

In [35]:
# define directory where baseline sketches will be saved out
save_dir = 'baseline_sketches'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [3]:
# mongo globals
conn = pm.MongoClient(port=20809)
DBNAME = conn['during_car_furniture_morphsketch']
COLNAME = DBNAME['baseline_physical.files']
coll=COLNAME

In [10]:
# define database connections -- how many workers?
db = conn['during_car_furniture_morphsketch']['baseline_physical.files']
dbe = conn['end_car_furniture_morphsketch']['baseline_physical']
workers = dbe.find({'WorkerID': {'$exists': True, '$ne':''}}).distinct('WorkerID')                                                       
workers = [w for w in workers if len(workers)>0]
print len(workers)

146


In [5]:
### List of workers & images that are not up to snuff
# good workers
good_workers = ['A1UNJWWSAB1D59','A1GXFMAC759VRM','A2ADEPVGNNXNPA']
# bad workers
bad_workers = ['A1A3TGZ7DKJWRW','A1EAEZ7VIHKQ2','A1A70T3F5D0A6E']
# bad sketch_id's (if there were cross outs, or blanks)
bad_ids = ['561297c18094844cae9f0728','5612975c8094844cae9f0718','561299c18094844cae9f076e','5611e4f78094844cae9f04b8','5612f00d8094844cae9f0aae','5612ed678094844cae9f0a94','561474228094844cae9f1f54']

In [31]:
## Render out furniture sketches
import urllib, cStringIO
furniture = ['bedChair', 'bedTable', 'benchBed', 'chairBench','chairTable', 'tableBench']
fs = gridfs.GridFS(conn['during_car_furniture_morphsketch'], 'baseline_physical')

## endpoint morphnums
morphlines = db.find({'$and': [{'morphline':{'$exists': True}}, {'category':'furniture'}]}).distinct('morphline')
all_morphnums = db.find({'$and': [{'morphline':{'$exists': True}}, {'category':'furniture'}]}).distinct('morphnum')
morphnums = ['0','99']
viewpoints = db.find({'$and': [{'viewpoint':{'$exists': True}}, {'category':'furniture'}]}).distinct('viewpoint')


In [40]:
records.count()

31

In [ ]:
for ml in morphlines:
    for mn in morphnums:
        for vp in viewpoints:
            records = db.find({'morphline':ml, 'morphnum':mn, 'viewpoint':vp}).sort('trialNum')
            print ml + ' ' + mn + ' ' + vp + ' ' + str(records.count())
            for i,r in enumerate(records):
                try:
                    imgData = r['imgData']
                    im = Image.open(cStringIO.StringIO(base64.b64decode(imgData)))
                    fig = plt.figure(figsize=(8,8))
                    p = plt.subplot(1,1,1)
                    plt.imshow(im)
                    k = p.get_xaxis().set_ticklabels([])
                    k = p.get_yaxis().set_ticklabels([])
                    k = p.get_xaxis().set_ticks([])
                    k = p.get_yaxis().set_ticks([])
                    for spine in plt.gca().spines.values():
                        spine.set_visible(False) 

                    obj_ind = get_object_index(ml,int(mn))
                    obj_name = get_object_name(2,obj_ind)
                    pathdir = os.path.join(save_dir)
                    pathfile = os.path.join(save_dir,'{}_{}_{}.png'.format(obj_name,vp,i))
                    fig.savefig(pathfile,bbox_inches='tight')  
                    print 'Saving to ' + pathfile
                    plt.close(fig)                    
                except:
                    print('Something went wrong with {} {} {}'.format(ml,mn,vp))
